In [6]:
#!/usr/bin/env conda run -n pymc3venv python

import sys
print(sys.executable)    ### Shoul be this C:\ProgramData\Anaconda3_1\envs\pymc3venv\python.exe

C:\ProgramData\Anaconda3_1\envs\pymc3venv\python.exe


In [7]:
# !conda install --yes --prefix {sys.prefix} tensorflow

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3_1\envs\pymc3venv

  added / updated specs:
    - tensorflow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _tflow_select-2.3.0        |              mkl           3 KB
    absl-py-0.9.0              |           py37_0         166 KB
    astor-0.8.0                |           py37_0          45 KB
    gast-0.3.3                 |             py_0          14 KB
    grpcio-1.27.2              |   py37h351948d_0         1.3 MB
    keras-applications-1.0.8   |             py_0          33 KB
    keras-preprocessing-1.1.0  |             py_1          36 KB
    libmklml-2019.0.5          |                0        21.4 MB
    libprotobuf-3.11.4         |       h7bd577a_0         2.2 MB
    markdown-3.1.1             |           py37_0         132 KB
    openssl-1.1.1d             |  

In [15]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import percentileofscore
from scipy import stats
from sklearn.model_selection import train_test_split
import h5py
import tensorflow as tf
from sklearn.metrics import classification_report

In [2]:
dfg = pd.read_hdf('dfg3.h5', key='losses')

In [4]:
dfg.describe()

,σ^v_[kPa],(N1)60,PI,LL,wn,CU-Vane,Pl_[Mpa],OCR,Cu-Tx
count,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000
mean,176.796699,15.250636,44.035301,74.122629,39.358225,251.525613,902.908709,3.749265,158.773322
std,82.191198,8.197895,10.845925,13.492986,7.172370,116.645540,345.763277,1.754469,55.567819
min,27.000000,5.081337,16.000000,31.000000,22.000000,23.000000,170.000000,1.920922,68.744449
25%,108.000000,10.019848,37.000000,66.850000,34.020000,175.230114,634.350000,2.414990,113.014801
50%,180.000000,12.667661,43.176923,74.000000,39.220833,235.000000,962.276295,3.000000,160.281449
75%,245.750000,18.314033,50.500000,82.116477,44.827232,319.771930,1162.918263,5.469882,204.752567
max,317.500000,67.681053,79.000000,109.000000,57.300000,530.000000,1700.000000,10.490107,263.541853


In [33]:
df = dfg.copy()
cols_to_norm = df.columns.tolist()
df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x-x.min())/(x.max()-x.min()))

In [34]:
df.columns = ['spv','N160','PI','LL','wn','CUvane','Plim','OCR','CuTx']

In [28]:
# transform CuTx in category
list = []
for i in df['CuTx']:
    if i <0.5:
        list.append(0)
    if i >=0.5:
        list.append(1)
df['CuTx']=list


In [29]:
df

,spv,N160,PI,LL,wn,CUvane,Plim,OCR,CuTx
3,0.000000,0.054611,0.398268,0.455711,0.203966,0.132150,0.202614,0.444527,0
4,0.000000,0.217552,0.435786,0.496503,0.215297,0.132150,0.202614,0.444527,0
5,0.000000,0.380492,0.473304,0.537296,0.226629,0.132150,0.202614,0.444527,0
6,0.003442,0.543432,0.510823,0.578089,0.226629,0.132150,0.202614,0.444527,0
7,0.015491,0.414341,0.548341,0.618881,0.226629,0.132150,0.202614,0.444527,0
...,...,...,...,...,...,...,...,...,...
460,0.994406,0.130071,0.539683,0.576923,0.564684,0.128205,0.509804,0.010289,1
461,0.997418,0.149969,0.539683,0.576923,0.593957,0.254109,0.480969,0.011349,1
462,0.998279,0.105435,0.539683,0.576923,0.623229,0.380013,0.452134,0.012410,1
463,1.000000,0.060901,0.536508,0.586325,0.594901,0.505917,0.423299,0.013471,1


In [30]:
## create feature columns ##

spv_feat = tf.feature_column.numeric_column('spv')
N160_feat = tf.feature_column.numeric_column('N160')
PI_feat = tf.feature_column.numeric_column('PI')
LL_feat = tf.feature_column.numeric_column('LL')
wn_feat = tf.feature_column.numeric_column('wn')
CUvane_feat = tf.feature_column.numeric_column('CUvane')
Plim_feat = tf.feature_column.numeric_column('Plim')
OCR_feat = tf.feature_column.numeric_column('OCR')


feat_columns = [spv_feat,N160_feat,PI_feat,LL_feat,wn_feat,CUvane_feat,Plim_feat,OCR_feat]

In [31]:
# Create test and train data

X_data = df.drop('CuTx',axis=1)
labels = df['CuTx']

X_train,X_test,y_train,y_test = train_test_split(X_data,labels,test_size=0.8,random_state=101)

In [32]:
# Define the Input Function

input_function = tf.estimator.inputs.pandas_input_fn(x=X_train,
                                                     y=y_train,
                                                     batch_size=10,
                                                     num_epochs=1000,
                                                     shuffle=True)
model = tf.estimator.LinearClassifier(feature_columns=feat_columns,n_classes=100)
model.train(input_fn=input_function, steps = 1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\fpe\\AppData\\Local\\Temp\\tmpo16ymeh3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002587F668A48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done call

In [45]:
# Prediction

pred_input_func = tf.estimator.inputs.pandas_input_fn(
                x=X_test,
                batch_size=10,
                num_epochs=1,
                shuffle=False)

predictions = model.predict(input_fn=pred_input_func)

final_preds=[]

for pred in predictions:
    final_preds.append(pred['class_ids'][0])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\fpe\AppData\Local\Temp\tmpo16ymeh3\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [48]:
# risultato

final_preds[:10]

[0, 1, 1, 1, 0, 0, 0, 1, 1, 1]

In [50]:
# Valori reali

y_test[:10].tolist()

[0, 0, 1, 1, 0, 0, 0, 1, 1, 1]

In [51]:
# Evaluate the model
# from sklearn.metrics import classification_report

print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           0       1.00      0.80      0.89       201
           1       0.80      1.00      0.89       169

    accuracy                           0.89       370
   macro avg       0.90      0.90      0.89       370
weighted avg       0.91      0.89      0.89       370

